In [1]:
import pandas as pd

In [5]:
cb = pd.read_parquet(r"Z:\dataset\Audit_Analytics\common\feed12_company_block.pq", columns=['COMPANY_FKEY', 'BEST_EDGAR_TICKER', 'CUSIP_NUMBER', 'ISIN'])
cb.dropna(subset=['BEST_EDGAR_TICKER', 'CUSIP_NUMBER', 'ISIN'], how='all', inplace=True)
cb.drop_duplicates(inplace=True)
cb.shape
cb.head(3)

(19854, 4)

,COMPANY_FKEY,BEST_EDGAR_TICKER,CUSIP_NUMBER,ISIN
1,1750,AIR,000361105,US0003611052
2,1800,ABT,002824100,US0028241000
6,1853,None,61980X202,None


In [6]:
lk = pd.read_parquet(r"Z:\dataset\Std_Security_Code\isin\cusip.pq")[['cusip', 'isin']]
lk.columns = ['CUSIP_NUMBER', 'ISIN2']
lk.shape
lk.head(3)

(616951, 2)

,CUSIP_NUMBER,ISIN2
63159,15135U10,CA15135U1093
56338,F5262B11,FR0004024222
56345,80603710,US8060371072


In [7]:
cb2 = cb.merge(lk, on='CUSIP_NUMBER', how='left')
cb2.shape
cb2.head(3)

(19855, 5)

,COMPANY_FKEY,BEST_EDGAR_TICKER,CUSIP_NUMBER,ISIN,ISIN2
0,1750,AIR,000361105,US0003611052,US0003611052
1,1800,ABT,002824100,US0028241000,US0028241000
2,1853,None,61980X202,None,US61980X2027


In [8]:
cb2.loc[cb2['ISIN'].isna(), 'ISIN'] = cb2['ISIN2']

In [12]:
lk = pd.read_parquet(r"Z:\dataset\Std_Security_Code\isin\ticker.pq")[['ticker', 'isin']]
lk.columns = ['BEST_EDGAR_TICKER', 'ISIN3']
lk.shape
lk.head(3)

(163013, 2)

,BEST_EDGAR_TICKER,ISIN3
0,TOU.J,CA89156V1067
73419,A030390,KR7030390009
73417,BRE2,SE0000648669


In [13]:
cb3 = cb2.merge(lk, on='BEST_EDGAR_TICKER', how='left')
cb3.shape
cb3.head(3)

(33277, 6)

,COMPANY_FKEY,BEST_EDGAR_TICKER,CUSIP_NUMBER,ISIN,ISIN2,ISIN3
0,1750,AIR,000361105,US0003611052,US0003611052,FR0000183790
1,1750,AIR,000361105,US0003611052,US0003611052,NL0000235190
2,1750,AIR,000361105,US0003611052,US0003611052,US0092791005


In [14]:
cb3.loc[cb3['ISIN'].isna(), 'ISIN'] = cb3['ISIN3']

# CUSIP

In [25]:
new = cb3[['CUSIP_NUMBER', 'ISIN']].drop_duplicates().dropna()
new.columns = ['cusip', 'isin']
new['source'] = 'Audit Analytics'
new['update'] = pd.Timestamp.now().date()
new.shape
new.sample(3)

(14661, 4)

,cusip,isin,source,update
22911,87166Q109,US87166Q1094,Audit Analytics,2024-05-05
23916,17039A106,CA17039A1066,Audit Analytics,2024-05-05
10932,00433U209,US00433U2096,Audit Analytics,2024-05-05


In [26]:
lk = pd.read_parquet(r"Z:\dataset\Std_Security_Code\isin\cusip.pq")
lk.shape
lk.head(3)

(616951, 4)

,isin,cusip,source,update
63159,CA15135U1093,15135U10,Refinitiv ESG,2022-06-17
56338,FR0004024222,F5262B11,Refinitiv ESG,2022-06-17
56345,US8060371072,80603710,Refinitiv ESG,2022-06-17


In [27]:
lkn = pd.concat([lk, new], ignore_index=True)
lkn['update'] = pd.to_datetime(lkn['update'])
lkn.sort_values(['isin', 'cusip', 'update'], ascending=True, inplace=True)
lkn.drop_duplicates(subset=['isin', 'cusip'], keep='last', inplace=True)
lkn.shape
lkn.head(3)

(620502, 4)

,isin,cusip,source,update
9614,AEDFXA0M6V00,M2851K10,Refinitiv ESG,2022-06-17
46321,AGP168751099,P41758106,Compustat NA,2022-06-16
8759,AGP8696W1045,P8696W10,Refinitiv ESG,2022-06-17


In [28]:
lkn['source'].value_counts()

source
FISD               561091
Compustat NA        34531
Audit Analytics     14661
Refinitiv ESG       10219
Name: count, dtype: int64

In [29]:
lkn.to_parquet(r"Z:\dataset\Std_Security_Code\isin\cusip.pq", index=False)

# Ticker

In [30]:
new = cb3[['BEST_EDGAR_TICKER', 'ISIN']].drop_duplicates().dropna()
new.columns = ['ticker', 'isin']
new['source'] = 'Audit Analytics'
new['update'] = pd.Timestamp.now().date()
new.shape
new.sample(3)

(10105, 4)

,ticker,isin,source,update
12066,BASWQ,US06985P2092,Audit Analytics,2024-05-05
3849,CFG,US1746101054,Audit Analytics,2024-05-05
15680,MLFB,US5608141058,Audit Analytics,2024-05-05


In [31]:
lk = pd.read_parquet(r"Z:\dataset\Std_Security_Code\isin\ticker.pq")
lk.shape
lk.head(3)

(163013, 4)

,isin,ticker,source,update
0,CA89156V1067,TOU.J,Refinitiv ESG,2022-06-17
73419,KR7030390009,A030390,S&P ESG,2022-06-17
73417,SE0000648669,BRE2,S&P ESG,2022-06-17


In [32]:
lkn = pd.concat([lk, new], ignore_index=True)
lkn['update'] = pd.to_datetime(lkn['update'])
lkn.sort_values(['isin', 'ticker', 'update'], ascending=True, inplace=True)
lkn.drop_duplicates(subset=['isin', 'ticker'], keep='last', inplace=True)
lkn.shape
lkn.head(3)

(164746, 4)

,isin,ticker,source,update
36707,AE0005802543,DIC,Sustainalytics,2022-06-17
36736,AE0005802550,DIB,Sustainalytics,2022-06-17
36200,AE0005802576,EMAAR,Sustainalytics,2022-06-17


In [33]:
lkn['source'].value_counts()

source
S&P ESG            91271
Compustat NA       45508
Refinitiv ESG      15250
Audit Analytics    10105
Sustainalytics      2612
Name: count, dtype: int64

In [34]:
lkn.to_parquet(r"Z:\dataset\Std_Security_Code\isin\ticker.pq", index=False)

# Fkey

In [36]:
new = cb3[['COMPANY_FKEY', 'ISIN']].drop_duplicates().dropna()
new.columns = ['company_fkey', 'isin']
new['source'] = 'Audit Analytics'
new['update'] = pd.Timestamp.now().date()
new['update'] = pd.to_datetime(new['update'])
new.drop_duplicates(subset=['company_fkey', 'isin'], keep='last', inplace=True)
new.shape
new.sample(3)

(18406, 4)

,company_fkey,isin,source,update
31309,1847806,SGXZ34583559,Audit Analytics,2024-05-05
16843,1353615,CA31863L1040,Audit Analytics,2024-05-05
18311,1398026,US87855G2066,Audit Analytics,2024-05-05


In [37]:
new.to_parquet(r"Z:\dataset\Std_Security_Code\isin\company_fkey.pq", index=False)